In [2]:
!mkdir -p ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d divaniazzahra/mental-health-dataset

Dataset URL: https://www.kaggle.com/datasets/divaniazzahra/mental-health-dataset
License(s): MIT
  0% 0.00/2.34M [00:00<?, ?B/s]
100% 2.34M/2.34M [00:00<00:00, 1.04GB/s]


In [6]:
!unzip mental-health-dataset.zip

Archive:  mental-health-dataset.zip
  inflating: Mental Health Dataset.csv  


In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [9]:
df=pd.read_csv("Mental Health Dataset.csv")
df.head()

,Timestamp,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,2014-08-27 11:29:31,Female,United States,Corporate,NaN,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,2014-08-27 11:31:50,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,2014-08-27 11:32:39,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,2014-08-27 11:37:59,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,2014-08-27 11:43:36,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292364 entries, 0 to 292363
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Timestamp                292364 non-null  object
 1   Gender                   292364 non-null  object
 2   Country                  292364 non-null  object
 3   Occupation               292364 non-null  object
 4   self_employed            287162 non-null  object
 5   family_history           292364 non-null  object
 6   treatment                292364 non-null  object
 7   Days_Indoors             292364 non-null  object
 8   Growing_Stress           292364 non-null  object
 9   Changes_Habits           292364 non-null  object
 10  Mental_Health_History    292364 non-null  object
 11  Mood_Swings              292364 non-null  object
 12  Coping_Struggles         292364 non-null  object
 13  Work_Interest            292364 non-null  object
 14  Social_Weakness     

In [12]:
df=df.drop("Timestamp",axis=1)

In [13]:
df

,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,Female,United States,Corporate,NaN,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292359,Male,United States,Business,Yes,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,Maybe,Not sure
292360,Male,South Africa,Business,No,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,Yes
292361,Male,United States,Business,No,Yes,No,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,No
292362,Male,United States,Business,No,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,Yes


In [19]:
mode_value=df["self_employed"].mode()[0]
df["self_employed"]=df["self_employed"].fillna(mode_value)

In [21]:
df.isnull().sum()

,0
Gender,0
Country,0
Occupation,0
self_employed,0
family_history,0
treatment,0
Days_Indoors,0
Growing_Stress,0
Changes_Habits,0
Mental_Health_History,0


In [22]:
df

,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,Female,United States,Corporate,No,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292359,Male,United States,Business,Yes,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,Maybe,Not sure
292360,Male,South Africa,Business,No,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,Yes
292361,Male,United States,Business,No,Yes,No,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,No
292362,Male,United States,Business,No,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,Yes


In [24]:
df.value_counts().sum()

np.int64(292364)

In [25]:
df["Gender"]=df["Gender"].map({"Female":0,"Male":1})

In [29]:
top_countries = df["Country"].value_counts().head(8).index
df['Country_grouped'] = df['Country'].apply(lambda x: x if x in top_countries else 'Other')


In [34]:
df["Country_grouped"].value_counts()
df.drop("Country",axis=1)

,Gender,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options,Country_grouped
0,0,Corporate,No,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure,United States
1,0,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No,United States
2,0,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes,United States
3,0,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes,United States
4,0,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292359,1,Business,Yes,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,Maybe,Not sure,United States
292360,1,Business,No,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,Yes,Other
292361,1,Business,No,Yes,No,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,No,United States
292362,1,Business,No,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,Yes,United States


In [43]:
df_country_dummies = pd.get_dummies(df["Country_grouped"], drop_first=False).astype(int)
df = pd.concat([df.drop("Country_grouped", axis=1), df_country_dummies], axis=1)


In [45]:
df.drop("Country",axis=1,inplace=True)

In [48]:
df_occupation_dummies = pd.get_dummies(df["Occupation"], drop_first=False).astype(int)
df = pd.concat([df.drop("Occupation", axis=1), df_occupation_dummies], axis=1)


In [51]:
df["self_employed"]=df["self_employed"].map({"Yes":1,"No"})

In [54]:
df["family_history"]=df["family_history"].map({"Yes":1,"No":0})

In [57]:
# df["treatment"].value_counts()
df["treatment"]=df["treatment"].map({"Yes":1,"No":0})

In [63]:
oe = OrdinalEncoder(categories=[["1-14 days", "15-30 days", "31-60 days", "More than 2 months", "Go out Every day"]])
df["Days_Indoors"] = oe.fit_transform(df[["Days_Indoors"]])

In [67]:
stress_mapping = {"No": 0, "Maybe": 1, "Yes": 2}
df["Growing_Stress"] = df["Growing_Stress"].map(stress_mapping)


In [70]:
changes_mapping = {"No": 0, "Maybe": 1, "Yes": 2}
df["Changes_Habits"] = df["Changes_Habits"].map(changes_mapping)


In [73]:
mh_history_mapping = {"No": 0, "Maybe": 1, "Yes": 2}
df["Mental_Health_History"] = df["Mental_Health_History"].map(mh_history_mapping)


In [76]:
mood_mapping = {"Low": 0, "Medium": 1, "High": 2}
df["Mood_Swings"] = df["Mood_Swings"].map(mood_mapping)

In [85]:
# Mapping dictionaries
binary_mapping = {"No": 0, "Yes": 1}
ordinal_3_mapping = {"No": 0, "Maybe": 1, "Yes": 2}
ordinal_3_not_sure_mapping = {"No": 0, "Yes": 1, "Not sure": 2}

# Apply mappings
df["Coping_Struggles"] = df["Coping_Struggles"].map(binary_mapping)
df["Work_Interest"] = df["Work_Interest"].map(ordinal_3_mapping)
df["Social_Weakness"] = df["Social_Weakness"].map(ordinal_3_mapping)
df["mental_health_interview"] = df["mental_health_interview"].map(ordinal_3_mapping)
df["care_options"] = df["care_options"].map(ordinal_3_not_sure_mapping)




,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,0,0,2,0,2
1,0,0,2,0,0
2,0,0,2,0,1
3,0,0,2,1,1
4,0,0,2,0,1


In [87]:
X=df.drop("treatment",axis=1)
Y=df["treatment"]
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [89]:
X_train.shape

(233891, 27)

In [102]:
model=Sequential()

model.add(Dense(64,activation="relu",input_dim=27))
model.add(Dense(128,activation="relu"))
model.add(Dense(256,activation="relu"))

model.add(Dense(1,activation="sigmoid"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [103]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 64)             │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,393 (169.50 KB)

 Trainable params: 43,393 (169.50 KB)

 Non-trainable params: 0 (0.00 B)

In [104]:
model.compile(loss="binary_crossentropy",optimizer=Adam(learning_rate=0.0005),metrics=["accuracy"])

In [105]:
history=model.fit(X_train,Y_train,epochs=20,batch_size=64,validation_split=0.2)

Epoch 1/20
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.6931 - loss: 0.5863 - val_accuracy: 0.7472 - val_loss: 0.5092
Epoch 2/20
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7566 - loss: 0.4864 - val_accuracy: 0.7607 - val_loss: 0.4675
Epoch 3/20
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7667 - loss: 0.4565 - val_accuracy: 0.7650 - val_loss: 0.4574
Epoch 4/20
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7675 - loss: 0.4512 - val_accuracy: 0.7664 - val_loss: 0.4589
Epoch 5/20
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7692 - loss: 0.4501 - val_accuracy: 0.7656 - val_loss: 0.4529
Epoch 6/20
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7696 - loss: 0.4463 - val_accuracy: 0.7647 - val_loss: 0.4513
Epoch 7/20
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7691 - loss: 0.4460 - val_accuracy: 0.7667 - val_loss: 0.4510
Epoch 8/20
2924/2924 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7673 - loss: 0.4463 -

In [109]:
y_pred_prob=model.predict(X_test)
y_predict = (y_pred_prob > 0.5).astype(int)


1828/1828 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


In [110]:
print(f"confusion matrix is: ",confusion_matrix(Y_test,y_predict))
print(f"Classification report is: ",classification_report(Y_test,y_predict))
print(f"Accuracy score is: ",accuracy_score(Y_test,y_predict))

confusion matrix is:  [[20551  8344]
 [ 5180 24398]]
Classification report is:                precision    recall  f1-score   support

           0       0.80      0.71      0.75     28895
           1       0.75      0.82      0.78     29578

    accuracy                           0.77     58473
   macro avg       0.77      0.77      0.77     58473
weighted avg       0.77      0.77      0.77     58473

Accuracy score is:  0.7687137653275871
